In [ ]:
#import packages

import numpy as np
import torch
import matplotlib.pyplot as plt


import os
import pandas as pd 
import random
import gc
import torch.cuda as cuda
from google.colab import drive
import torchvision as tv
import torchvision.transforms as transforms
from torchvision import models
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import ImageFile

drive.mount('/content/drive')


In [ ]:


!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
EFNet = EfficientNet.from_pretrained('efficientnet-b3')
print(EFNet)
EFNet._fc = torch.nn.Sequential()#1536
for  name,param in EFNet.named_parameters():
  if ("bn" not in name):
    param.requires_grad = False
EFNet.set_swish(memory_efficient=False)


In [3]:
class mlp(torch.nn.Module):
  def  __init__(self,input_size, output_num):
    super(mlp,self).__init__()
    self.fc1=torch.nn.Linear(input_size,output_num)
 
    self.relu=torch.nn.ReLU()
    self.drop4= torch.nn.Dropout(p=0.8)
    
    
  def forward(self,x):
   
    out=self.drop4(x)
    out=self.fc1(out)

    return out

In [4]:
model=mlp(1536 , 150)

In [5]:
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=(400,350), scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=20),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225]) 
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize(size=(400,350)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
data = {
    'train':
    datasets.ImageFolder(root="/content/drive/My Drive/dog/batch5/train_dir/", transform=image_transforms['train']),
    'valid':
    datasets.ImageFolder(root="/content/drive/My Drive/dog/batch5/valid_dir/", transform=image_transforms['valid']),
    'test':
    datasets.ImageFolder(root="/content/drive/My Drive/dog/test_dir/", transform=image_transforms['valid']),
}


In [7]:
train_samples_lenght = len(data['train'])
valid_samples_lenght = len(data['valid'])
test_samples_lenght = len(data['test'])

batch_size_train = 25
batch_size_valid = 15
numTrainBatches =  int(np.ceil(train_samples_lenght / batch_size_train))
numValidBatches =  int(np.ceil(valid_samples_lenght / batch_size_valid))
numTestBatches =  int(np.ceil(test_samples_lenght / batch_size_valid))


dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size_train, shuffle=True),
    'val': DataLoader(data['valid'], batch_size=batch_size_valid, shuffle=False),
    'test': DataLoader(data['test'], batch_size=batch_size_valid, shuffle=False)

}

In [ ]:
print(numTrainBatches)
print(numValidBatches)
print(numTestBatches)

In [9]:
def cal_accuracy(pred_classes,real_classes):#  shape ->  both : [number of test samples,]
  bool_array=(pred_classes==real_classes) # example: bool_array=[True, False, True, False, True]
  True_pred_counts=np.count_nonzero(bool_array) # count number of Trues in [True, False, True, False, True]
  return True_pred_counts/pred_classes.shape[0]

In [10]:
criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), betas=(0.9, 0.999), eps=1e-08,lr=0.0001)


In [ ]:
minloss= 0.3
train_loss=0
test_loss=0
for i in range(150):
  ac=0
  train_loss=0
  test_loss=0
  trainiter = iter(dataloaders['train']) 
  for k in range(numTrainBatches):
    x, y = next(trainiter) #features: torch.Size( batch_size, 3 ,400, 350) , labels: torch.Size(batch_size)



    if cuda.is_available():
          x = x.cuda()
          y = y.cuda()

          EFNet = EFNet.cuda()
          model = model.cuda()
    
    EFNet.eval()
    model.train()
    
    x = EFNet(x)
    outputs=model(x) #output: torch.Size([batchsize, 150])  | 150 is number of output neurons
    
    
    loss=criterion(outputs,y)
    train_loss += loss.data

    optimizer.zero_grad()   
    loss.backward()

    pred_classes=torch.max(outputs.data, 1)[1]
    acc=cal_accuracy(np.array(pred_classes.cpu()),np.array(y.cpu()))
    ac=acc+ac



    del outputs
    del loss
    del x
    del y
    del pred_classes

    gc.collect()

    optimizer.step()

    torch.cuda.memory_allocated()
    torch.cuda.empty_cache() 

  print("Epoch: ",i+1," Train loss=",train_loss/numTrainBatches," Accuracy: ",ac*100 / numTrainBatches)
  gc.collect()
  ac=0
  validiter = iter(dataloaders['val']) 
  for k in range(numValidBatches):
    xx, yy = next(validiter) #features: torch.Size( batch_size, 3 ,400, 350) , labels: torch.Size(batch_size)
    

    if cuda.is_available():
          xx = xx.cuda()
          yy = yy.cuda()

          EFNet = EFNet.cuda()
          model = model.cuda()


    EFNet.eval()
    model.eval()

    xx = EFNet(xx)
    outputss=model(xx) # output: torch.Size([batchsize, 150])  | 150 is number of output neurons

    loss=criterion(outputss,yy)
    test_loss=test_loss+loss.data
    pred_classes=torch.max(outputss.data, 1)[1]
    acc=cal_accuracy(np.array(pred_classes.cpu()),np.array(yy.cpu()))
    ac=acc+ac


    del outputss
    del loss
    del xx
    del yy
    gc.collect()
    torch.cuda.empty_cache() 
  print("Epoch: ",i+1," Test loss=",test_loss/numValidBatches," Accuracy: ",ac*100 / numValidBatches)
  if (test_loss/numValidBatches)< minloss:
    print("hopLoss")
    minloss=test_loss/numValidBatches
    PATH="/content/drive/My Drive/dog/net5.pth"
    torch.save({

              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
  
              }, PATH)
       
  
 
print(ac) 

In [ ]:
### TEST ###
model=mlp(1536 , 150)
PATH="/content/drive/My Drive/dog/net5.pth"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.cuda()

test_loss=0
ac = 0 
testiter = iter(dataloaders['test']) 
for k in range(numTestBatches):
    xx, yy = next(testiter) #features: torch.Size( batch_size, 3 ,400, 350) , labels: torch.Size(batch_size)
    

    if cuda.is_available():
          xx = xx.cuda()
          yy = yy.cuda()

          EFNet = EFNet.cuda()
          model = model.cuda()


    EFNet.eval()
    model.eval()

    xx = EFNet(xx)
    outputss=model(xx) # output: torch.Size([batchsize, 150])  | 150 is number of output neurons

    loss=criterion(outputss,yy)
    test_loss=test_loss+loss.data
    pred_classes=torch.max(outputss.data, 1)[1]
    acc=cal_accuracy(np.array(pred_classes.cpu()),np.array(yy.cpu()))
    ac=acc+ac


    del outputss
    del loss
    del xx
    del yy
    gc.collect()
    torch.cuda.empty_cache() 
print(" Test loss=",test_loss/numTestBatches," Accuracy: ",ac*100 / numTestBatches)